# Retention Score Test - Redshift
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Mar 23, 2022

In [2]:
import boto3
from datetime import datetime
from termcolor import colored, cprint
import pandas as pd
import awswrangler as wr
import pyodbc

### First we have to tell AWS which profile we'd like to login to, this will send us for a browser authentication trip:

In [3]:
! aws sso login --profile Stellaralgo-DataScienceAdmin

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

HBCN-DNSX
Successully logged into Start URL: https://stellaralgo.awsapps.com/start


### Connect to Redshift:

In [4]:
try:
    session = boto3.setup_default_session(profile_name='Stellaralgo-DataScienceAdmin')
    client = boto3.client('redshift')

    conn = wr.data_api.redshift.connect(
        cluster_id="qa-app",
        database="datascience",
        db_user="admin"
    )
    
    print('CREDENTIALS RETRIEVED SUCCESSFULLY')
    
except Exception as err:
    print('FAILED TO RETRIEVE CREDENTIALS!', err)

CREDENTIALS RETRIEVED SUCCESSFULLY


In [5]:
sqlcode =  f"""
    SELECT DISTINCT
        clientcode,
        lkupclientid,
        dbname,
        leagueName
    FROM 
        datascience.ds.teamsconfig
    """
df = wr.data_api.redshift.read_sql_query(
            sql = sqlcode, 
            con = conn
        )

columns = df.columns.values

teams = []
for index, row in df.iterrows():
    teams.append(dict(zip(columns, row)))


print(f"Total teams: {len(teams)} ")
print(f"Example team {teams[0]} ")

Total teams: 39 
Example team {'clientcode': 'hops', 'lkupclientid': 9, 'dbname': 'stlrmilb', 'leaguename': 'MILB'} 


### Create a dictionary of all teams that have retention scoring:

In [6]:
for team in teams:
    
    conn = wr.data_api.redshift.connect(
        cluster_id="qa-app",
        database= team['dbname'],
        db_user="admin"
    )

    sql = f"""
            SELECT TOP 1 insertdate, count(*) as count
            FROM {team['dbname']}.dw.customerretentionscores  
            WHERE [lkupclientid] = {team['lkupclientid']}
            GROUP BY insertdate 
            ORDER BY convert(datetime, insertdate) DESC
        """

    df_customerretention = wr.data_api.redshift.read_sql_query(
        sql = sql, 
        con = conn
    )
    
    for row in df_customerretention.iterrows():
        team['insertdate'] = row[1]['insertdate']
        team['count'] = row[1]['count']
        
print(f"Example team {teams[0]} ")

Example team {'clientcode': 'hops', 'lkupclientid': 9, 'dbname': 'stlrmilb', 'leaguename': 'MILB', 'insertdate': '2022-04-01 10:23:16', 'count': 931} 


### Write a test to see if insertdate is greater than todays date:

In [7]:
today = datetime.now().strftime("%m-%d-%Y 00:00:00")

print("SCORE REPORT FOR DATASCIENCE customerScores TABLE:")
for team in teams:

    try:
        if team['insertdate'] > today:
            print(colored(f" > {team['count']} SCORES UPDATED: {team['clientcode']} ({team['lkupclientid']})", 'green'))
        else:
            print(colored(f" ! {team['count']} SCORES NOTE UPDATED: {team['clientcode']} ({team['lkupclientid']})", 'yellow'))
    
    except Exception as err:
        print(colored(f" ! NO RETENTION SCORES EXIST: {team['clientcode']} ({team['lkupclientid']})", 'red'))


SCORE REPORT FOR DATASCIENCE customerScores TABLE:
 > 931 SCORES UPDATED: hops (9)
 > 3581 SCORES UPDATED: bulls (11)
 ! NO RETENTION SCORES EXIST: rivercats (12)
 > 1755 SCORES UPDATED: vegas51s (15)
 > 111 SCORES UPDATED: rainiers (17)
 > 247 SCORES UPDATED: 66ers (19)
 > 603 SCORES UPDATED: loons (20)
 > 408 SCORES UPDATED: rattlers (24)
 > 16 SCORES UPDATED: legends (26)
 ! NO RETENTION SCORES EXIST: grizzlies (27)
 > 325 SCORES UPDATED: fireflies (28)
 > 931 SCORES UPDATED: elpaso (30)
 > 902 SCORES UPDATED: rrexpress (34)
 > 1591 SCORES UPDATED: renoaces (40)
 > 384 SCORES UPDATED: stormchasers (43)
 ! NO RETENTION SCORES EXIST: hartfordyardgoats (44)
 > 1762 SCORES UPDATED: okcdodgers (47)
 > 2296 SCORES UPDATED: drive (48)
 > 814 SCORES UPDATED: stripers (49)
 > 249 SCORES UPDATED: kanecounty (51)
 > 1042 SCORES UPDATED: toledomudhens (59)
 > 5651 SCORES UPDATED: stamps (35)
 > 1276 SCORES UPDATED: roughnecks (8)
 > 777 SCORES UPDATED: hitmen (37)
 > 9 SCORES UPDATED: nhlpanthe